In [1]:
from pathlib import Path
import pandas as pd
from Bio import Entrez, SeqIO
import time
import re

Entrez.email = "yourmail.com"

In [2]:
data_path = Path.cwd().parent/'datas'

vali_df = pd.read_csv(data_path / "extra_vali.csv", index_col=0)

sequenceID = vali_df.index.tolist()

# Based on the gene ID, download the corresponding fasta to a single file.

In [63]:
# This is a function that downloads multiple FASTA sequences and writes them into a single file.
#
# The function `download_fasta` takes two parameters:
# 1. `sequence_ids`: A list or iterable containing IDs of the nucleotide sequences to be fetched from NCBI's Entrez database.
# 2. `filename`: The name of the output file where the downloaded FASTA sequences will be concatenated.
def download_fasta(sequence_ids, filename):
    """
    Download mutiple FASTA sequences.
    
    Args:
    sequence_ids (list): List of IDs representing the sequences to fetch from NCBI's nucleotide database.
    filename (str): The name of the output file to which the sequences will be written.

    """
    with open(filename, 'a') as output_file: 
        for seq_id in sequence_ids:
            with Entrez.efetch(db='nucleotide', id=seq_id, rettype="fasta", retmode="text") as handle: 
                seq_data = handle.read()
                output_file.write(seq_data)
                output_file.write('\n') 
                
            time.sleep(1) 

In [68]:
# Download the corresponding sequence according to sequenceID
download_fasta(sequenceID, data_path / "extra_validata.fa")

In Linux, using seqkit delate duplicated sequences

# Find the virus name and ID correspondence

In [4]:
# Extract Sequence IDs and virus names.
def parse_fasta_header(fasta_file):
    """
    Parse sequence ID and virus name.

    Args:
    fasta_file (str): Path to the input FASTA file.

    Returns:
    list[tuple]: A list of tuples, where each tuple contains the parsed sequence ID and the corresponding virus name.

    """

    # Initialize an empty list to store the results
    data = []
    
    # Match sequence ID and virus name
    pattern = re.compile(r'^(\S+)\s(.*?virus)')

    for seq_record in SeqIO.parse(fasta_file, "fasta"):
        header = seq_record.description
        match = pattern.match(header)
        if match:
            seq_id, virus_name = match.groups()
            data.append((seq_id, virus_name))
        else:
            print(f"Cannot match header: {header}")

    return data

In [40]:
# Parse the fasta file to get the virus id and name
file_path = data_path / "zoontic_prediction" / "extra_validata_rmd.fa"

data = parse_fasta_header(file_path)

df = pd.DataFrame(data, columns=['SequenceID', 'Name'])

Cannot match header: MZ244241.1 MAG: Huangpi Tick Virus 1 isolate SXO334peribunya nucleocapsid protein gene, complete cds


In [41]:
# manually add the above virus names and IDs

temp_data = ['MZ244241.1', 'Huangpi Tick Virus 1']

new_index = len(df)

df.loc[new_index] = temp_data

# Locate the virus cds area

In [61]:
def fetch_cds_details(seq_id):
    """
    Fetch CDS (Coding Sequence) details from GenBank records.

    Parameters:
    seq_id (str): The sequence ID for which CDS details are to be fetched.

    Returns:
    list: A list of tuples containing sequence ID, start, and end positions of each CDS.
    
    """

    # Initialize an empty list to store details of CDS (Coding Sequences)
    cds_details = []

    # Retrieve GenBank records
    handle = Entrez.efetch(db="nucleotide", id=seq_id, rettype="gb", retmode="text")
    record = SeqIO.read(handle, "genbank")
    handle.close()

    # Extract CDS information
    for feature in record.features:
        if feature.type == "CDS":
            is_complement = feature.location.strand == -1

            start = int(feature.location.start) + 1 # must + 1
            end = int(feature.location.end)

            cds_details.append((seq_id, start, end, is_complement))
            
    return cds_details

In [62]:
# Initialize an empty list to store the CDS details for all sequences
all_cds = []

# fetch each sequence CDS details
for seq_id in sequenceID:
    cds_details = fetch_cds_details(seq_id)
    all_cds.extend(cds_details)
    
    time.sleep(0.5)

# transform to DataFrame
cds_df = pd.DataFrame(all_cds, columns=['SequenceID', 'CodingStart', 'CodingStop', 'converse'])

In [90]:
# Transpose the complementary chain
cds_df.loc[cds_df['converse'], ['CodingStart', 'CodingStop']] = cds_df.loc[cds_df['converse'], ['CodingStop', 'CodingStart']].values

# Changing Sequence ID
cds_df['SequenceID'] = cds_df['SequenceID'].apply(lambda x: x if x.endswith('.1') else x + '.1')

# combining name and cds data
zoonotic_data = pd.merge(cds_df, df, on='SequenceID', how='left')

cols = ['Name'] + [col for col in zoonotic_data.columns if col != 'Name']

zoonotic_data = zoonotic_data[cols]

zoonotic_data = zoonotic_data.drop("converse", axis=1)

Since the MK896599.1 sequence updates the sequence to MK896599.2, change it manually.

In [84]:
zoonotic_ids = zoonotic_data['SequenceID'].tolist()
df_ids = df['SequenceID'].tolist()

file_path = data_path / "zoontic_prediction" / "extra_validata.fa"
fasta_ids = [record.id for record in SeqIO.parse(file_path, 'fasta')]


[id for id in zoonotic_ids if id not in fasta_ids]

['MK896599.1']

changing MK896599.1 to MK896599.2

In [91]:
zoonotic_data.loc[zoonotic_data['SequenceID'] == "MK896599.1", 'SequenceID'] = "MK896599.2"

zoonotic_data.to_csv(data_path / "zoontic_prediction" / "zoonotic_metadata.csv", index=None)

In [56]:
zoonotic_data

,Name,SequenceID,CodingStart,CodingStop
0,Caimito virus,NC_055408.1,70,804
1,Caimito virus,NC_055410.1,53,4429
2,Caimito virus,NC_055409.1,57,6731
3,Estero Real orthobunyavirus,NC_055216.1,61,4269
4,Estero Real orthobunyavirus,NC_055217.1,12,11789
...,...,...,...,...
280,Ebinur lake virus,NC_079005.1,86,787
281,Ebinur lake virus,OR861624.1,1,2208
282,Ebinur lake virus,KJ710423.1,53,4360
283,Ebinur lake virus,KJ710424.1,86,787
